In [12]:
# Educational Shor implementation (small N only)
# - Builds controlled-U^(2^j) gates from explicit permutation matrices.
# - Runs quantum phase estimation (QPE) and classical postprocessing.
#
# WARNING: Works only for small N (e.g., N=15,21...). Not suitable for real RSA integers.

import math
import numpy as np
from fractions import Fraction
from sympy import continued_fraction, continued_fraction_reduce  # for rational approx
from qiskit import QuantumCircuit ,transpile
#from qiskit.extensions import UnitaryGate
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

# -------------------------
# Utilities: math helpers
# -------------------------
def is_coprime(a, b):
    return math.gcd(a, b) == 1

def find_a_coprime(N):
    # choose a random a in [2, N-1] s.t. gcd(a, N) = 1
    import random
    candidates = [x for x in range(2, N) if math.gcd(x, N) == 1]
    if not candidates:
        return None
    return random.choice(candidates)

def continued_fraction_to_rational(x, max_denominator):
    # use Python's Fraction.limit_denominator as simpler alternative
    frac = Fraction(x).limit_denominator(max_denominator)
    return frac.numerator, frac.denominator

# -------------------------
# Build permutation unitary for multiply-by-a mod N
# -------------------------
def multiplication_unitary_matrix(a, N, m):
    """
    Build 2^m x 2^m permutation matrix U where:
      U |x> = |a * x mod N>  for x in 0..N-1
      U |x> = |x> for x >= N   (leave "overflow" states fixed)
    Returns a numpy array (complex dtype).
    """
    dim = 2**m
    U = np.zeros((dim, dim), dtype=complex)
    for x in range(dim):
        if x < N:
            y = (a * x) % N
            U[y, x] = 1.0
        else:
            # keep states >= N unchanged
            U[x, x] = 1.0
    return U

# -------------------------
# Quantum Phase Estimation circuit (manual)
# -------------------------
def build_qpe_circuit(U_matrices, t, m):
    """
    Build the QPE circuit.
    - U_matrices: list of numpy matrices [U^(1), U^(2), U^(4), ...] each size 2^m x 2^m,
      one for each counting qubit (i.e. U^(2^j)).
    - t: number of counting qubits
    - m: number of qubits in computational register
    """
    total_qubits = t + m
    qc = QuantumCircuit(total_qubits, t)

    # 1) Put counting register in uniform superposition
    for q in range(t):
        qc.h(q)

    # 2) Prepare computational register in state |1> (commonly used if 1 is a good start)
    # Put 1 in the least-significant qubit of the computational register (index t)
    qc.x(t + 0)  # set |...0001> ; note ordering: we index computational qubits 0..m-1 as t..t+m-1

    # 3) Controlled applications of U^(2^j)
    for j in range(t):
        U = U_matrices[j]
        U_gate = UnitaryGate(U)
        # create controlled version with one control
        cU = U_gate.control(1)
        # control qubit is j (counting register), target is computational register (t .. t+m-1)
        qc.append(cU, [j] + [t + k for k in range(m)])
        # Note: qiskit's ordering for apply of UnitaryGate(control) works with the assigned list

    # 4) Inverse QFT on counting register
    qc = qc.compose(inverse_qft(t), list(range(t)))  # apply inverse QFT to first t qubits

    # 5) Measure counting register
    for i in range(t):
        qc.measure(i, i)

    return qc

# -------------------------
# Inverse QFT (on n qubits)
# -------------------------
def inverse_qft(n):
    qc = QuantumCircuit(n)
    # Build inverse QFT (with swaps to correct bit order)
    for j in range(n//2):
        qc.swap(j, n - j - 1)
    for j in range(n):
        for k in range(j):
            qc.cp(-2 * math.pi / (2 ** (j - k + 1)), k, j)  # controlled phase with negative angle
        qc.h(j)
    return qc

# Alternative more standard inverse QFT implementation (safer numerical ordering)
def inverse_qft(n):
    qc = QuantumCircuit(n)
    # Apply the inverse QFT (standard decomposition, no final swaps because we will interpret bits accordingly)
    for qubit in range(n//2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        # apply controlled phase gates from higher-index qubits to lower-index
        for k in range(j):
            angle = -math.pi / float(2 ** (j - k))
            qc.cp(angle, k, j)
        qc.h(j)
    return qc

# -------------------------
# Classical postprocessing: get order r from measured phase
# -------------------------
def get_r_from_counts(counts, t, N, a, shots=1024):
    """
    Analyze measurement result histogram `counts` (dict mapping bitstring->freq)
    Return candidate r or None.
    """
    # pick the most probable outcome
    measured = max(counts.items(), key=lambda kv: kv[1])[0]  # bitstring (MSB..LSB)
    # Qiskit's measurement result is string with qubit 0 as leftmost; we used counting qubits 0..t-1
    # Interpret measured as integer
    k = int(measured, 2)
    phase = k / (2 ** t)

    # Convert phase to fraction with denominator up to N (search)
    num, den = continued_fraction_to_rational(phase, N)
    r_candidate = den
    if r_candidate == 0:
        return None

    # verify r is indeed order: a^r mod N == 1
    if pow(a, r_candidate, N) == 1:
        return r_candidate
    # sometimes need to try multiples or factors of den
    # try denominators from continued fraction convergents (use sympy continued fractions)
    cf = continued_fraction(phase, 20)
    for i in range(1, len(cf)+1):
        frac = continued_fraction_reduce(cf[:i])
        num_i, den_i = int(frac.as_numer_denom()[0]), int(frac.as_numer_denom()[1])
        if den_i == 0: 
            continue
        if den_i <= 0:
            continue
        if den_i > N:
            continue
        if pow(a, den_i, N) == 1:
            return den_i
    return None

# -------------------------
# Main Shor routine (small N)
# -------------------------
def shor_factoring_small(N, a=None, t=None, shots=256):
    """
    Attempt to factor N using a small-N Shor demonstration (explicit matrices).
    - Choose a (coprime to N) randomly if not provided.
    - t: number of counting qubits (precision); default ~ 2*m
    Returns factors or None.
    """
    if N < 2:
        raise ValueError("N must be >= 2")
    # choose a coprime a
    if a is None:
        a = find_a_coprime(N)
    if not is_coprime(a, N):
        raise ValueError("a must be coprime to N")
    print(f"Attempting to factor N={N} with a={a}")

    # determine m (number of qubits for computational register)
    m = int(math.ceil(math.log2(N)))
    dim = 2 ** m
    if dim < N:
        raise ValueError("Need 2^m >= N")

    # choose t (number of counting qubits) if not provided
    if t is None:
        t = 2 * m  # common heuristic

    # build U^(2^j) matrices for j=0..t-1
    # first build U (multiply by a mod N)
    U = multiplication_unitary_matrix(a, N, m)
    U_powers = []
    # U^(2^j) = U ^ (2^j) (matrix power)
    for j in range(t):
        power = 2 ** j
        Uj = np.linalg.matrix_power(U, power)
        U_powers.append(Uj)

    # build circuit
    qc = build_qpe_circuit(U_powers, t, m)

  
    #Introduce Simulator
    simulator=AerSimulator()


    # Transpile the circuit for the simulator
    compiled_circuit = transpile(qc, simulator)

    # Run the circuit on the simulator
    job = simulator.run(compiled_circuit, shots=1000)

    # Get the results
    result = job.result()
    counts = result.get_counts(qc)
    print(f"Measurement counts: {counts}")


    # classical postprocessing: get r
    r = get_r_from_counts(counts, t, N, a, shots=shots)
    print("Estimated r:", r)
    if r is None:
        print("Failed to find order r; try again with different a or more shots/t.")
        return None

    # if r is odd, often fails — try r's factors
    if r % 2 == 1:
        print("Found odd r; need even r. Try another a.")
        return None

    # try to compute factors
    x = pow(a, r // 2, N)
    p = math.gcd(x - 1, N)
    q = math.gcd(x + 1, N)
    if p in (1, N) or q in (1, N):
        print("Trivial factors found; try another a.")
        return None

    return (p, q)

# -------------------------
# Example run
# -------------------------
if __name__ == "__main__":
    # Small examples: N = 15, 21, 33...
    for N in [15, 21]:
        print("\n---\nFactoring N =", N)
        # try a few random a's if first attempt fails
        success = None
        for attempt in range(6):
            a_choice = find_a_coprime(N)
            try:
                res = shor_factoring_small(N, a=a_choice, t=6, shots=512)
            except Exception as e:
                print("Error:", e)
                res = None
            if res:
                print(f"Success: factors of {N} = {res}")
                success = res
                break
            else:
                print(f"Attempt {attempt+1} with a={a_choice} failed; retrying.")
        if not success:
            print(f"Could not factor {N} in the attempts. Try increasing t or shots, or choose other a.")



---
Factoring N = 15
Attempting to factor N=15 with a=14
Measurement counts: {'100000': 501, '000000': 499}
Estimated r: 2
Trivial factors found; try another a.
Attempt 1 with a=14 failed; retrying.
Attempting to factor N=15 with a=11
Measurement counts: {'000000': 509, '100000': 491}
Error: continued_fraction() takes 1 positional argument but 2 were given
Attempt 2 with a=11 failed; retrying.
Attempting to factor N=15 with a=4
Measurement counts: {'000000': 494, '100000': 506}
Estimated r: 2
Success: factors of 15 = (3, 5)

---
Factoring N = 21
Attempting to factor N=21 with a=19
Measurement counts: {'101010': 23, '110110': 39, '001010': 34, '101011': 110, '110101': 108, '100010': 1, '001011': 121, '100000': 163, '110111': 6, '000000': 170, '110100': 6, '000011': 1, '101100': 6, '010101': 98, '101110': 1, '000101': 1, '110010': 1, '001001': 6, '010100': 7, '100011': 1, '001100': 7, '010110': 35, '011101': 1, '000100': 1, '101101': 3, '110011': 4, '100100': 1, '001101': 2, '001000': 3